# Symmetric Cryptography and AES

## MATH 157 (Winter 23) - Final Project

**Tomás Gillanders**  
**PID: A17694974**

Project Available at: [GitHub - TomasGillanders](https://github.com/TomasGillanders/MATH157FinalProject)

In [1]:
using Pkg
# PKG.add("LinearAlgebra")
# Pkg.add("Nemo")
# Pkg.add("Primes")
# Pkg.add("AES")

# Import the Libraries used by the project.
# using LinearAlgebra;
# using Nemo;
# using Primes;
# using AES;

In [2]:
# Include Files with Constants Defined
include("./AESTables.jl");

## Introduction

### §0.1 Symmetric Cryptography

The concept of *Symmetric Cryptography* is simpler to that of *Asymmetric Cryptography*, as was discussed in class. With Symmetric Cryptography, the same *key* is used in both the encryption and decryption processes.

Suppose that we have two people, Alice and Bob, who want to exchange messages without a third-party, Eve say, being able to read what they are sending.  Alice and Bob decide that they want to use a *symmetric* crytographic system to encode/encrypt their messages, so that the only thing Eve can see is their scrambled data.  To do this, Alice and Bob decide on a *key* that they can use to encrypt and decrypt their messages.  They can then send messages to each other with ease, knowing that their secrets are safe, as long as their mutual key is kept secret from everbody else.

![Symmetric Encryption](./Images/SymmetricEncryption.png)

The use of *Symmetric Cryptography* has many advantages:
- The production of strong keys is generally computionally less expensive, when compared to keys used in asymmetric cryptography.
- The size of the keys used is generally much smaller than keys used in asymmetric crytography, while providing comparable levels of protection.
- The algorithms implemented are generally computationally inexpensive.

However, there are also some significant drawbacks with using a Symmetric Cryptosystem:
- If a third-party (Eve) were to find out the secret key, it would allow them to both decrypt messages and also encrypt new messages and insert them into Alice and Bob's conversation; neither of whom would know that the message was not sent by the other.
- How Alice and Bob share their key initially is also problematic.  If they cannot meet in person (as is the case in reality), then they must rely on a different secret key to protect the privacy of the key that they are trying to share.  However, this means that they must have the other secret key in the first place, and how was that key shared?  This can lead to a never-ending dependency on a different key.



### §0.2 The Advanced Encryption Standard (AES)

***The Advanced Encryption Standard (AES)*** is a *symmetric block cipher* that developed between 1997 and 2001 by the *US Institute of Standards and Technology* (NIST) to become a *Federal Information Processing Standard* (FIPS) and replace the now redundant *Data Encryption Standard* (DES).

NIST opened the development of candidate symmetric ciphers to independent parties, who's algorithms were then compared and scrutinized publically in order to determine who's algorithm provided the best balance between security, efficiency and the ability to be implemented over different platforms; ranging from 8-bit processors to servers and work-stations.

The algorithm eventually chosen and adopted was a slight adaptation of an algorithm developed by Joan Daemen and Vincent Daemen, known as *Rijndael*.*

AES is now used by the US Government to protect, confidential, secret and top secret information, with longer cryptographic keys being used when encrypting top-secret information, for added security. [reference here: TechTarget]

The mathematical theory behind AES and its implementation is presented in this project.
- In Part 1, we discuss the mathematical theory of finite fields, that underpins the AES algorithm.
- In Part 2, we discuss the method of encryption, and build functions to do so.
- In Part 3, we define both the encryption and decryption functions, using our building blocks from Part 2.
- In Part 4, we apply our encryption and decryption functions to plaintext examples.

## Part 1: Finite Fields - Theory

### § 1.1 Laying the Foundations

***Definition 1.1.1*** **(Abelian Group)**: Let $G$ be a non-empty set on which there is defined a binary operation $+: G \times G \rightarrow G$.  We say that $(G,+)$ is a group iff the following axioms hold:
- Closure:           $\qquad \qquad \forall a,b \in G, \; a+b \in G$
- Associativity:     $\qquad \ \  \forall a,b,c \in G, \; (a+b)+c = a+(b+c)$
- Identity Element:  $\quad \  \exists e \in G, \; \forall a \in G, \; a + e = e + a = a$
- Inverses:          $\qquad \quad \ \ \ \ \forall a \in G, \; \exists (-a) \in G, \; a + (-a) = (-a) + a = e$
- Commutivity:       $\qquad \ \ \forall a,b \in G, \; a+b = b+a$

For an abelian group, the operation, $+$, is often reffered to as addition, and the identity element is denoted by $\textbf{0}$.

***Definition 1.1.2*** **(Ring)**: Let $R$ be a non-empty set on which there is defined two binary operations, denoted $+,\bullet: R \times R \rightarrow R$.  We say that $(R, +, \bullet \,)$ is a ring iff:
1. The algebraic structure $(R, +)$ constitutes an abelian group.
2. The operation, $\bullet$, is closed, associative and there exists an identity element (i.e. $(R,\bullet \,)$ constitutes a monoid).
3. Distributivity:   $ \qquad \ \ \  \forall a,b,c \in R: \; a \bullet(b+c) = a \bullet b + a \bullet c $

We say that $(R,+,\bullet \,)$ is a commutative ring if the operation, $\bullet$, is commutative.  The identity element for $\cdot$ is often denoted by $\textbf{1}$ (while the identity element for $+$ is denoted by $\textbf{0}$, as above).

***Definition 1.1.3*** **(Field)**: Let $F$ be a non-empty set on which there is defined two binary operations $+,\bullet: F \times F \rightarrow F$.  We say that $(F, +, \bullet)$ is a field iff:
1. $(F, +, \bullet)$ is a commutative ring.
2. $\forall a \in F \setminus \{\textbf{0}\}, \; \exists a^{-1} \in F \setminus \{\textbf{0}\}$ such that $a \bullet a^{-1} = a^{-1} \bullet a = \textbf{1}$.


---
---

### §1.2 Finite Fields

In the implementation of AES/Rijndael, the theory of Finite (Galois) Fields plays a central role in the encryption and decyrption processes.


***Definition 1.2.1*** **(Finite Field)**: A Finite Field is a field with a finite number of elements.  The number of elements in the set is called the ***order*** of the field.

>*Note: When discussing fields for the remainder of this project, it is implicite that the fields in question are finite, unless otherwise stated.*

***Theorem 1.2.2*** *(Existence of Finite Fields)*: For every prime $p$ and positive integer $m$, there exists a finite field of order $p^m$.  We say that $p$ is the ***characteristic*** of the field.

***Theorem 1.2.3***: All finite fields of the same order are *isomorphic*. [isomorphic reference].

*Theorem 1.2.3* tells us that that while (finite) fields of the same order may differ in the representation of their elements, allgebraically their structures are identical.  Therefore, combining *Theorems 1.2.2* and *1.2.3*, we have that for every prime power, there exists exactly one finite field.  We denote the Finite (Galois) Field of order $p^n$ by:

$$ GF(p^n) $$

---
---





### §1.3 Operations on Finite Fields & Polynomial Representations

#### Fields of Prime Order:

Consider the field $GF(p)$, where $p$ is a prime. The elements of $GF(p)$ can be represented by the integers 

$$ 0, 1, 2, ..., p-2, p-1 $$ 

Then the two operations of field addition and multiplication on $GF(p)$ are the familiar operations of integer addition modulo $p$ and integer multiplication modulo $p$. 

---

### Paricipation Check

Consider the field $GF(17)$.  Using the `Nemo` library, or otherwise, find $a,b,c \in GF(17)$, where
1. $a = 15 + 5$
2. $b = 13 \bullet 3$
3. $10 = 13 \bullet c$

In [3]:
# YOUR ANSWER HERE
# ...

# Answers:
F17 = GF(17)

@show a = F17(15) + 5
@show b = F17(13) * 3
@show c = F17(10) // 13;  # or use `powermod()` without Nemo

LoadError: UndefVarError: GF not defined

---

#### Fields of Non-Prime Order:

Consider the field $GF(p^n)$, with $n>1$.  For such fields the operations of addition and multiplication cannot be defined/represented in terms of integer addation and multiplication as was done above.

> **Example**: Consider $GF(4)$.  *Theorem 1.2.2* tells us that $GF(4)$ is a field (with the appropriate field operations), since $4 = 2^2$.  However, if we consider the element $2 \in GF(4)$, we note that 
>
>$$ \forall n \in \mathbb{Z}: \quad 2 \bullet n \equiv 0 \pmod{4} \quad \text{or} \quad 2 \bullet n \equiv 2 \pmod{4} $$
>
>Thus, there does not exist $2^{-1} \in \mathbb{Z}$ such that $2 * 2^{-1} \equiv 1 \pmod{4}$.  This means that integer multiplication modulo 4 cannot be one of the operations defined on $GF(4)$.



In [4]:
# Check for the first 1001 non-negative integers
ints = collect(0:1000)

1 ∈ (2 .* ints) .% 4

false

In order to define our field operations on $GF(p^n)$, $n>1$, we move to representing its elements in terms of polynomials, rather than the integers $0,1,...,p-1$.

***Definition 1.3.1*** **(Polynomial over a Field)**:  Let $F$ be a field.  A polynomial over $F$ is an expression of the form:

$$ p(x) = \sum_{i = 0}^{n-1} a_i x^i $$

where $x$ is called the ***indeterminate*** of the polynomial and $a_i \in F$ are the ***coefficients***.

***Definition 1.3.2*** **(Degree of Polynomial)**: The ***degree*** of a polynomial is the least $l$ such that $a_j = 0, \; \forall j > l$.  

We denote by $F[x]$ the *ring* of all polynomials over $F$, and by $F[x]|_l$ the *set* of all polynomials over $F$ with degree strictly less than $l$.

---

Using *Definition 1.3.1* we can now represent the elements of $GF(p^n)$ as follows.  Consider an integer $b$ in the range $0,1,...,p^n-1$:

- First write $b$ with respect to the base $p$.  This gives us a list of terms:

    $$ (b)_p = (b_{n-1} b_{n-2} ... b_2 b_1 b_0)_p $$
    where
    $$ b = b_{n-1} \cdot p^{n-1} + ... + b_1 \cdot p^1 + b_0 \cdot p^0 $$ 
    
    
- Then, since each $b_i$ is in the range $0,1,...,p-1$, we can identify $b$ with a unique polynomial in $GF(p)|_n$ as follows:

$$ b \longleftrightarrow b(x) =  b_{n-1} x^{n-1} + ... +  b_2 x^2 + b_1 x + b_0$$ 

---

We can now define our operations on $GF(p^n)$ as follows:

Consider two elements $a,b \in GF(p^n)$.  We identify $a,b$ with polynomials in the field $GF(p)[x]|_n$:

$$ a \longleftrightarrow a(x) \qquad \text{and} \qquad b \longleftrightarrow b(x) $$

We recall that by *Theorem 1.2.3*, $GF(p^n)$ and $GF(p)[x]|_n$ are algebraicly the same.* [see appendix on discussion of $GF(p)[x]|_n$ as a field.] 

- **Addition**: $a + b \longleftrightarrow a(x) + b(x)$ where the addition of the coefficients of like powered terms in the polynomial sum occurs in $GF(p)$.
- **Multiplication**: We note that for $GF(p)[x]|_n$ to be closed under polynomial multiplication an irreducible ***reduction polynomial*** of degree $n$, $m(x)$, must be chosen.  Multiplication is then defined in terms of:

$$ a \bullet b \longleftrightarrow a(x) \bullet b(x) \pmod{m(x)} $$

---

\* see appendix/link/book... on discussion of $GF(p)[x]|_n$ as a field.

---

### §1.4 Example: Applying Finite Fields and AES

In the implementation of AES, bits of data are grouped into bytes.  Therefore, the set of all possible byte-values can be thought of as elements (polynomials) of the field $GF(2)[x]|_8$ or, as discussed above, $GF(2^8)$.

Thus, when looking to encrypt a given plaintext byte, the operations and theory of finite fields can be applied.

In the specification of AES, bytes are considered as polynomials , with 'addition' and 'multiplication' of bytes as discussed above.  We note also that the following irreducible polynomial is used as reduction polynomial for all multiplication operations:

$$ m(x) = x^8 + x^4 + x^3 + x + 1 $$

Note that while the byte multiplication is used in the AES specification, no computation of byte multiplication is required in its implmentation.  This is further discussed in §2.(...).


> **Byte Addition and its Implemenation**:
>
> Consider the elements $53, 12 \in GF(2^8)$.  We wish to find the sum $53 + 12 \in GF(2^8)$. We carry out this addition as follows:


In [5]:
# Using the Nemo library intialise the Field GF(2) and the Polynomial Ring GF(2)[x]
@show GF2        = GF(2)
@show R_GF2_x, x = GF2["x"];

LoadError: UndefVarError: GF not defined

In [6]:
# Find the binary representations of 53 and 12.
@show bin53 = string(53, base = 2)
@show bin12 = string(12, base = 2);

bin53 = string(53, base = 2) = "110101"
bin12 = string(12, base = 2) = "1100"


In [7]:
# Parse and return as vectors.
@show vec53 = parse.(Int, split(bin53, ""))
@show vec12 = parse.(Int, split(bin12, ""));

vec53 = parse.(Int, split(bin53, "")) = [1, 1, 0, 1, 0, 1]
vec12 = parse.(Int, split(bin12, "")) = [1, 1, 0, 0]


In [8]:
# Produce the polynomial representations
@show pol53 = R_GF2_x(reverse(vec53))
@show pol12 = R_GF2_x(reverse(vec12));

LoadError: UndefVarError: R_GF2_x not defined

In [9]:
# Add the polynomials over GF(2)[x]
@show pol53 + pol12;

LoadError: UndefVarError: pol53 not defined

> Thus, we have that the addition of 53 and 12 over $GF(2^8)$ corresponds to the number with binary representation
> $$ 111001 $$
> We use the `parse()` function again to parse this number in decimal.

In [10]:
@show answer = parse(Int, "111001" , base = 2);

answer = parse(Int, "111001", base = 2) = 57


> Thus, we have that in $GF(2^8)$, $53 + 12 = 57$.
>
> However, on closer examination of the above method, we note that the operation in fact simply implemented, without consideration of polynomial addition. 

---

### Participation Check

Compute $157 + 200$ in $GF(2^8)$, without using the polynomial addition method above. (This can be accomplished in one short line of code).

In [11]:
# Your Answer Here



**Answer**: Looking at the above example, we notice that addition of elements of $GF(2^8)$ is simply the operation of taking the ***Bitwise XOR*** of the two elements.  Julia allows us to apply this operation using the $\veebar$ (\xor - tab) symbol.

*Note*: Often the symbol $\oplus$ is used to represent XOR in literature.  However, here we use $\veebar$ to remain consistent with the Julia syntax.

In [12]:
@show 157 ⊻ 200;

157 ⊻ 200 = 85


---

---
---
---

## Part 2: The Encryption Algorithm

### §2.1 Overview

AES is a ***Key-Iterated Block Cipher*** that acts on a square $4 \times 4$ array of bytes (this is how the plaintext data is grouped).

***Definition 2.1.1*** **(Block Cipher)**: A *Block Cipher* is a function that operates on a *Plaintext Block* of fixed length (number of bits), and returns a *ciphertext block* of the same length, under the influence of a cipher key, $\bf{k}$.

An ***iterative block cipher*** is a block cipher in which some function(s) (called ***Boolean Permutations***) are repeatedly applied to the given ***state*** of the block of data. These Boolean Permutations are called the ***Round Transformations***, and every application of a Round Transformation is called a ***Round***.

Each Round is dependent on a ***Round Key***, which is generated using the original cipher key (discussed later).  If we call $\bf{k}^{(i)}$ the $i^{th}$ round key, with $\bf{k}^{(0)}$ the original cipher key, then the concatenation of all round keys is called the ***Expanded Key***, denoted $K$:

$$ K = \bf{k}^{(0)}|\bf{k}^{(2)}|\bf{k}^{(3)}|...|\bf{k}^{(r)} $$

where $r$ is the number of round applications.

If we take $\rho$ to be the round transformation, $\sigma[k^{(i)}]$ to be the byte addition (bitwise XOR) of the current *state* with the $i^{th}$ round key and $B[k]$ to be the block cipher, then AES takes the form:

$$ B[K] = \sigma[\bf{k}^{(r)}] \circ \rho^\star \circ \sigma[\bf{k}^{(r-1)}] \circ \rho \circ \cdots \sigma[\bf{k}^{(1)}] \circ \rho \circ \sigma[\bf{k}^{(0)}] $$

where $\rho^\star$ is the final round, which differs slightly.

We now discuss the structure of each round.

### §2.2 The Round Tranformation, `Round`

In discussing the round transformation of the AES Encryption Algorithm, we follow the naming conventions as layed-out in Daemen and Rijmen's "The Design of Rijndael; AES - The Advanced Encryption Standard". [insert reference]



#### §2.2.1 `SubBytes`

`SubBytes` is a "non-linear transformation"* that acts byte-wise on the current *state*.  Such a transformation is called a ***Bricklayer Permutation***, with its defining characteristic being that the function can be decomposed into a number of ***Boolean Permutations*** that act on subsets of bits.  In the case of `SubBytes`, the bits in the current *state* are partitioned into bytes.  We note that such a transformation is invertible, since each of the costituent *Boolean Permutations*, acting independently, are invertible.

The *Boolean Permutions* in question are called ***S-boxes***, if non-linear, and ***D-boxes***, if linear.

`SubBytes` is an *S-box* that acts on the bytes of the current state.  It consists of the composition of a non-linear permutation and affine transformation on $GF(2^8)$:
- The non-linear transformation is simlpy the mapping of an element in $GF(2^8)$ to its inverse:

$$ g:GF(2^8) \rightarrow GF(2^8), \; a \mapsto b = a^{-1} $$

This operation is carried out using the polynomial representation of $GF(2^8)$, as discussed in §1.4; using the irreducible polynomial $m(x) = x^8 + x^4 + x^3 + x + 1$ as the reduction polynomial.
    
- The invertible affine transformation is defined as follows:

$$ f(a) = b \iff \begin{bmatrix} b_7 \\
                   b_6 \\
                   b_5 \\
                   b_4 \\
                   b_3 \\
                   b_2 \\
                   b_1 \\
                   b_0
                   \end{bmatrix} =  \begin{bmatrix} 1 & 1 & 1 & 1 & 1 & 0 & 0 & 0 \\
                   0 & 1 & 1 & 1 & 1 & 1 & 0 & 0 \\
                   0 & 0 & 1 & 1 & 1 & 1 & 1 & 0 \\
                   0 & 0 & 0 & 1 & 1 & 1 & 1 & 1 \\
                   1 & 0 & 0 & 0 & 1 & 1 & 1 & 1 \\
                   1 & 1 & 0 & 0 & 0 & 1 & 1 & 1 \\
                   1 & 1 & 1 & 0 & 0 & 0 & 1 & 1 \\
                   1 & 1 & 1 & 1 & 0 & 0 & 0 & 1 \\
                   \end{bmatrix} \bullet \begin{bmatrix} 
                   a_7 \\
                   a_6 \\
                   a_5 \\
                   a_4 \\
                   a_3 \\
                   a_2 \\
                   a_1 \\
                   a_0
                   \end{bmatrix} \veebar \begin{bmatrix} 
                   0 \\
                   1 \\
                   1 \\
                   0 \\
                   0 \\
                   0 \\
                   1 \\
                   1 \\\end{bmatrix}
$$
 
The *S-box* is then the function $f \circ g(a)$.


While the above operations underpin the `SubBytes` operation, when implementing the encryption algorithm, no computation of inverses or matrix multiplication is required.  In stead, a lookup table is used, far improving the efficiency of the algorithm.

In [13]:
transpose(reshape(SBOX,16,16))

16×16 transpose(::Matrix{UInt8}) with eltype UInt8:
 0x63  0x7c  0x77  0x7b  0xf2  0x6b  …  0x67  0x2b  0xfe  0xd7  0xab  0x76
 0xca  0x82  0xc9  0x7d  0xfa  0x59     0xa2  0xaf  0x9c  0xa4  0x72  0xc0
 0xb7  0xfd  0x93  0x26  0x36  0x3f     0xe5  0xf1  0x71  0xd8  0x31  0x15
 0x04  0xc7  0x23  0xc3  0x18  0x96     0x80  0xe2  0xeb  0x27  0xb2  0x75
 0x09  0x83  0x2c  0x1a  0x1b  0x6e     0xd6  0xb3  0x29  0xe3  0x2f  0x84
 0x53  0xd1  0x00  0xed  0x20  0xfc  …  0xbe  0x39  0x4a  0x4c  0x58  0xcf
 0xd0  0xef  0xaa  0xfb  0x43  0x4d     0x02  0x7f  0x50  0x3c  0x9f  0xa8
 0x51  0xa3  0x40  0x8f  0x92  0x9d     0xda  0x21  0x10  0xff  0xf3  0xd2
 0xcd  0x0c  0x13  0xec  0x5f  0x97     0x7e  0x3d  0x64  0x5d  0x19  0x73
 0x60  0x81  0x4f  0xdc  0x22  0x2a     0xb8  0x14  0xde  0x5e  0x0b  0xdb
 0xe0  0x32  0x3a  0x0a  0x49  0x06  …  0xac  0x62  0x91  0x95  0xe4  0x79
 0xe7  0xc8  0x37  0x6d  0x8d  0xd5     0xf4  0xea  0x65  0x7a  0xae  0x08
 0xba  0x78  0x25  0x2e  0x1c  0xa6     0x74  0x

<!-- <img src="./Images/SBox.png" width="70%" height="auto"> -->

Note that in the above lookup table, the elements of $GF(2^8)$ are expressed in hexadecimal.

The Inverse Transformation, `InvSubBytes`, is the *S-box* $(f \circ g)^{-1} = g^{-1} \circ f^{-1}$, and is similarly implmemented using a lookup table.

In [14]:
# SBOX and INVSBOX are defined in `AESTables.jl`.

# In place of defining the function `SubBytes`, we define the function `SubByte` that 
# acts on individual bytes.  This can then be broadcast across arrays and also allows
# for its implimentation within other functions.

function SubByte(a)
    # Add 1 to the index because the arrays are 1 indexed.
    return SBOX[Int(a+1)]
end

function InvSubByte(b)
    return INVSBOX[Int(b+1)]
end

InvSubByte (generic function with 1 method)

In [15]:
# Initialize a test state
teststate = rand(UInt8, 4,4)

4×4 Matrix{UInt8}:
 0x56  0xbb  0xfc  0x3b
 0xbc  0xe4  0x3c  0x1f
 0xb1  0x89  0x7c  0x6f
 0xb3  0xfd  0xb5  0xac

In [16]:
# Demonstation
InvSubByte.(SubByte.(teststate)) == teststate

true

#### §2.2.2 `ShiftRows`

`ShiftRows` is a *Boolean Permutation* that cyclically shifts each of the rows in the current state over different offsets.  By convention for AES:
- Row 0 is shifted (cyclically) 0 bytes to the left,
- Row 1 is shifted 1 byte to the left,
- Row 2 is shifted 2 bytes to the left, and
- Row 3 is shifted 3 bytes to the left.

##### Diffusion

The above offsets are selected as to optimize the ***diffusion*** effects of the transformation.  The *diffusion* of a transformation describes the "quantitative spreading of information". [insert reference, book]  By maximizing the *diffusion* of the transformation, over repeated implementations, bits in the ciphertext should depend on a "significant number" of bits in the plaintext (i.e. by changing a bit of the plaintext, a large proportion of the ciphertext bits should change).  *Diffusion* ensures that patterns that are apparent (or hidden) in the plaintext are not repeated in the ciphertext, and thus, cannot be exploited by malicious parties.

The inverse transformation, `InvShiftRows`, is simply obtained by composing `ShiftRows` with itself three times. By then applying `ShiftRows` followed by `InvShiftRows` (or vice versa) each row is offset by some integer multiple of 4.  Due to the cyclic nature of the shifts, the composition ends up being the identity.


In [17]:
# Implementation of `ShiftRows`
function ShiftRows(state)
    for i = 2:4
        # Copy row from state
        temp = state[i,:]
        
        # Cycle the rows
        for j = 1:(i-1)
            push!(temp, popfirst!(temp))
        end
        
        # Replace the row
        state[i,:] = temp
    end
    return state
end

# Repeatedly Implment `ShiftRows`
# 1•4, 2•4, 3•4 ≡ 0 (mod 4)
InvShiftRows(state) = (ShiftRows ∘ ShiftRows ∘ ShiftRows)(state)  # Using the Function Composition Operator, `∘`


InvShiftRows (generic function with 1 method)

In [18]:
# Demonstration
InvShiftRows(ShiftRows(teststate)) == teststate

true

#### §2.2.3 `MixColumns`

Like the `SubBytes` transformation, `MixColumns` is also a *bricklayer permutations* but differed in that it acts on column rather than individual bytes, and that the tranformation is linear, and is thus, a *D-box*.

The tranformation in question considers the 4-byte columns of the current *state* as polynomials over $GF(2^8)$ with degree less than or equal to 3.  The multiplication used is then performed modulo the reduction polynomial:

$$ n(x) = x^4 + 1 $$

We note that even though by the discussion in §1.3, $n(x)$ is reducible, only multiplication by a constant polynomial, $c(x)$, that does have an inverse*, $d(x) \pmod{n(x)}$, is used in the implementation of `MixColumns`. 

Givin the column $a$ in the current state, where

$$ a = \begin{bmatrix} a_0 \\
                       a_1 \\
                       a_2 \\
                       a_3 \end{bmatrix} \iff a(x) = a_3 \cdot x^3 + a_2 \cdot x^2 + a_1 \cdot x + a_0
$$

`MixColumns` multiplies the polynomial representation of $a$ by the fixed polynomial

$$ c(x) = \texttt{0x03} \cdot x^3 + \texttt{0x01} \cdot x^2 + \texttt{0x01} x + \texttt{0x02} $$

and reduces the answer modulo $n(x)$.

*Note*: In the above polynomial, the coefficients are expressed in hexadecimal (using the same syntax as Julia).

Working out the product $b(x) = a(x) \bullet c(x)$ and reducing it modulo $n(x)$, we find that the multiplication can be expressed as a matrix-vector multiplication:

$$ \begin{bmatrix} b_0 \\
                   b_1 \\
                   b_2 \\
                   b_3 
                   \end{bmatrix} = 
                   \begin{bmatrix} \texttt{0x02} & \texttt{0x03} & \texttt{0x01} & \texttt{0x01} \\
                                   \texttt{0x01} & \texttt{0x02} & \texttt{0x03} & \texttt{0x01} \\
                                   \texttt{0x01} & \texttt{0x01} & \texttt{0x02} & \texttt{0x02} \\
                                   \texttt{0x03} & \texttt{0x01} & \texttt{0x01} & \texttt{0x02}
                                   \end{bmatrix} \otimes \begin{bmatrix} a_0 \\
                                                    a_1 \\
                                                    a_2 \\
                                                    a_3 \end{bmatrix}
                                                    $$

Where we use the symbol $\otimes$ to remind us that the matrix multiplication is being carried out over the field $GF(2^8)$.

In order to perform the operation of multiplication over $GF(2^8)$, we must define the function `mul`.  The implemenation of `mul` is explored in the accompanying homework exercise.

---
\* $d(x)$ is such that $c(x) \bullet d(x) \equiv 1 \pmod{n(x)}$

---

In [19]:
# FOR INFORMATION ON HOW `mul` IS IMPLEMENTED, SEE HOMEWORK PROBLEMS
function mul(a,b)
    prod::UInt8 = 0
    for _ = 1:8
        if (b & 1) != 0
            prod = prod ⊻ a
        end
        overflow = ((a & 0x80) != 0)
        a = a << 1
        if overflow
            a = a ⊻ 0x1b
        end
        b = b >> 1
    end
    prod
end

# Define `MixColumns`.  
# Note that the Inverse Tranformation can be completed by imputting the inverse transform matrix: `InvMixMat`
function MixColumns(state, c)
    for i = 1:4
        newtemp = zeros(UInt8, 4)
        tmp = state[:,i]
        
        for j = 1:4
            newtemp[j] = mul(tmp[1], c[j,1]) ⊻ mul(tmp[2], c[j,2]) ⊻ mul(tmp[3], c[j,3]) ⊻ mul(tmp[4], c[j,4]) 
        end
        
        state[:,i] = newtemp
    end
    return state
end


MixColumns (generic function with 1 method)

In [20]:
# Demonstration
MixColumns(MixColumns(teststate, InvMixMat), MixMat) == teststate

true

#### §2.2.4 `AddRoundKey`

`AddRoundKey` is the simplest of all the steps of the *Round Transformation*.  It consists of simply adding the current state and the generated *Round Key* using a bitwise XOR.

By nature of the bitwise XOR, `AddRoundKey` is its own inverse.

***Note***: Below, the function `KeyExpansion` is used.  This function is used to generate the round keys from the cipher key.  This function is defined in the "AESTables.jl" file that is included with this project.  Information on how the round keys are generated using `KeyExpansion` is explored in the accompanying homework questions.

In [21]:
function AddRoundKey(state, ExpandedKey, R)
    cols = 4R + 1
    return state .⊻ ExpandedKey[:,cols:(cols+3)]
end

AddRoundKey (generic function with 1 method)

In [22]:
# Demonstation

k = [
    0x2b 0x7e 0x15 0x16
    0x28 0xae 0xd2 0xa6
    0xab 0xf7 0x15 0x88
    0x09 0xcf 0x4f 0x3c
]

testkey = KeyExpansion(k)

AddRoundKey(AddRoundKey(teststate, testkey, 5), testkey, 5) == teststate

true

---
---
---

# Part 3: Encryption and Decription

The AES algorithm is structured such that the above *Round Tranformations* are applied iteratively to the supplied *plaintext* block, subject to the symmetric key that is chosen.





### § 3.1 The Encryption Function

AES encryption is structured as follows:
1. The first step of the encryption process is an an initial Key Addition:

$$ \texttt{AddRoundKey} $$

2. The *Round Transoformation*, `Round`, is then applied $N_r-1$ times, where $N_r$ is the total number of *rounds* that are to be applied to the *plaintext*.  `Round` has the following structure:

$$ \texttt{Round}: \quad \texttt{SubBytes} \longrightarrow \texttt{ShiftRows} \longrightarrow \texttt{MixColumns} \longrightarrow \texttt{AddRoundKey} $$

3. The final *Round Tranformation*, `FinalRound`, is the same as `Round`, but with the `MixColumns` step removed:

$$ \texttt{FinalRound}: \quad \texttt{SubBytes} \longrightarrow \texttt{ShiftRows} \longrightarrow  \texttt{AddRoundKey} $$

The resulting *ciphertext* is then returned by the encryption function, and can be shared to those who also have the symmetrix key, without worry of it being read/interpreted by intermediate parties.

The encryption function, `AESEncrypt`, is defined below.

In [23]:
function AESEncrypt(plaintext, cipherkey)
    # Define Variables Used
    N_rounds = 10
    state = plaintext
    
    # Generate `ExpandedKey`
    ExpandedKey = KeyExpansion(cipherkey)
    
    # Initial Key Addition
    state = AddRoundKey(state, ExpandedKey, 0)
    
    # Normal Rounds
    for i = 1:(N_rounds-1)
        state = SubByte.(state)
        state = ShiftRows(state)
        state = MixColumns(state, MixMat)
        state = AddRoundKey(state, ExpandedKey, i)
    end
    
    # Final Round
    state = SubByte.(state)
    state = ShiftRows(state)
    state = AddRoundKey(state, ExpandedKey, N_rounds)
end

AESEncrypt (generic function with 1 method)

In [24]:
# Demonstration
cipher = AESEncrypt(teststate, k)

4×4 Matrix{UInt8}:
 0x65  0x90  0x48  0xf6
 0x2c  0xad  0xaa  0xa2
 0xd7  0xee  0x89  0xdc
 0x20  0x19  0x1b  0x9b

---

### § Decryption Function

The AES *Decryption Algorithm* is easy to define, given the definitions of the *Round Transformations*.  Since each of the round tranformations are defined such that they are invertible, we can simply treat them as invertible mathematical functions, and use their properties under composition:

$$
\begin{align}\texttt{Round}^{-1} \quad = \quad &(\texttt{AddRoundKey} \circ \texttt{MixColumns} \circ \texttt{ShiftRows} \circ \texttt{SubBytes})^{-1}\\
= \quad  &\texttt{SubBytes}^{-1} \circ \texttt{ShiftRows}^{-1} \circ \texttt{MixColumns}^{-1} \circ \texttt{AddRoundKey}^{-1} \\ \\
\texttt{FinalRound}^{-1} \quad = \quad &(\texttt{AddRoundKey} \circ \texttt{ShiftRows} \circ \texttt{SubBytes})^{-1}\\
= \quad  &\texttt{SubBytes}^{-1} \circ \texttt{ShiftRows}^{-1} \circ \texttt{AddRoundKey}^{-1}
\end{align}
$$
 
  
$$
\begin{align}
\implies &\texttt{InvRound}: \qquad \quad \; \texttt{AddRoundKey} \longrightarrow \texttt{MixColumns} \longrightarrow \texttt{ShiftRows} \longrightarrow \texttt{SubBytes} \\
\implies &\texttt{InvFinalRound}: \quad \texttt{AddRoundKey} \longrightarrow \texttt{ShiftRows} \longrightarrow \texttt{SubBytes}
\end{align}
$$

We can therefore, decrypt the ciphertext by first applying `InvFinalRound` to it, followed by repeatedly applying `InvRound` for the appropriate number of rounds and finally applying the key addition `AddRoundKey`.  With each key additition performed, we make sure that we are round keys in the reverse order that they were applied for encryption.

***Note***:
- The transformations `InvShiftRows` and `InvSubBytes` commute, since one acts on rows of the current *state*, while the other acts on inidividual bytes, independent of their position.  Therefore, the order in which `InvShiftRows` and `InvSubBytes` doesn't matter.


- The tranformations `InvMixColumns` and `AddRoundKey` can be "inverted" as follows.  We note that the polynomial representations of the elements of $GF(2^8)$ can be associated with *vectors* in the *vector space* $GF(2)^8$, over the field $GF(2)$.  We then note that `MixColumns` is a *linear tranformation*, $D:GF(2)^8 \rightarrow GF(2)^8$ and `AddRoundKey` consists only of an addition of the current state, $\textbf{x} \in  GF(2)^8$ with a constant vector (the round key), $\textbf{k} \in GF(2)^8$.  Then by definition of a linear tranformation, we have:

$$ D(\textbf{x} \oplus \textbf{k}) = D(\textbf{x}) \oplus D(\textbf{k}) = D(\textbf{k}) \oplus D(\textbf{x}) $$ 

Thus, by first applying `InvMixColumns` to both the state and round key first and then applying the resulting transformed round key, the order of the operations can be "reversed".

However, while this abstraction is nice mathematically, for simplicity we just apply `InvMixColumns` and `AddRoundKey` in the order as stated above.

The decryption function, `AESDecrypt`, is defined below.


In [25]:
function AESDecrypt(ciphertext, cipherkey)
    # Define Variables
    N_rounds = 10
    state = ciphertext
    
    # Generate `ExpandedKey`
    ExpandedKey = KeyExpansion(cipherkey)
    
    # Last Round Inverse
    state = AddRoundKey(state, ExpandedKey, N_rounds)
    state = InvSubByte.(state)
    state = InvShiftRows(state)
    
    # Normal Rounds Inverse
    for i in reverse(1:9)
        state = AddRoundKey(state, ExpandedKey, i)
        state = MixColumns(state, InvMixMat)
        state = InvSubByte.(state)
        state = InvShiftRows(state)
    end
    
    # End with the Key Addition
    AddRoundKey(state, ExpandedKey, 0)
end

AESDecrypt (generic function with 1 method)

In [26]:
AESDecrypt(cipher, k) == teststate

true

---
---
---

# Part 4: Applying the Algorithm

### §4.1 Short Example

In [27]:
# Demonstration
str = "Mathematics=Fun!"
@show str_chars = only.(split(str, ""))
str_block = reshape(str_chars, 4,4)

str_chars = only.(split(str, "")) = ['M', 'a', 't', 'h', 'e', 'm', 'a', 't', 'i', 'c', 's', '=', 'F', 'u', 'n', '!']


4×4 Matrix{Char}:
 'M'  'e'  'i'  'F'
 'a'  'm'  'c'  'u'
 't'  'a'  's'  'n'
 'h'  't'  '='  '!'

In [28]:
plaintext = UInt8.(str_block)

4×4 Matrix{UInt8}:
 0x4d  0x65  0x69  0x46
 0x61  0x6d  0x63  0x75
 0x74  0x61  0x73  0x6e
 0x68  0x74  0x3d  0x21

In [29]:
cipher = AESEncrypt(plaintext, k)
cipher_str = (Char.(vec(cipher)))

16-element Vector{Char}:
 'â': Unicode U+00E2 (category Ll: Letter, lowercase)
 '\u92': Unicode U+0092 (category Cc: Other, control)
 'á': Unicode U+00E1 (category Ll: Letter, lowercase)
 '\u96': Unicode U+0096 (category Cc: Other, control)
 '/': ASCII/Unicode U+002F (category Po: Punctuation, other)
 '?': ASCII/Unicode U+003F (category Po: Punctuation, other)
 'o': ASCII/Unicode U+006F (category Ll: Letter, lowercase)
 'd': ASCII/Unicode U+0064 (category Ll: Letter, lowercase)
 'o': ASCII/Unicode U+006F (category Ll: Letter, lowercase)
 'y': ASCII/Unicode U+0079 (category Ll: Letter, lowercase)
 '\x03': ASCII/Unicode U+0003 (category Cc: Other, control)
 '\u9e': Unicode U+009E (category Cc: Other, control)
 '¯': Unicode U+00AF (category Sk: Symbol, modifier)
 '\u93': Unicode U+0093 (category Cc: Other, control)
 'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)
 'ô': Unicode U+00F4 (category Ll: Letter, lowercase)

In [30]:
decrypted = AESDecrypt(cipher, k)
decrypted_message = String(Char.(vec(decrypted)))

"Mathematics=Fun!"

---

### §4.2 Longer Examples

In [31]:
println(HumptyDumpty)

Humpty Dumpty sat on a wall.
Humpty Dumpty had a great fall.
All the king's horses and all the king's men
Couldn't put Humpty together again.


In [32]:
println(Lorem)

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Curabitur condimentum vitae tortor nec congue. Ut felis est, posuere viverra ex id, pharetra pulvinar odio. Nulla placerat mi sed pharetra efficitur. Curabitur a faucibus elit, vitae laoreet augue. Nulla finibus, orci non porta gravida, tellus lectus scelerisque lorem, ut iaculis turpis enim nec velit. In porttitor arcu ac justo scelerisque, et venenatis quam posuere. Nulla iaculis, lectus vitae fermentum tristique, urna neque vestibulum tellus, id congue dui nulla eget tellus. Mauris et turpis iaculis arcu convallis consectetur ac ac purus. Praesent fringilla hendrerit lacus, vitae blandit purus elementum vel. Donec vehicula arcu felis, non vulputate libero ultrices at. Fusce posuere, augue at imperdiet pulvinar, neque tortor dictum elit, nec sollicitudin neque sem eu leo. Sed eget accumsan nisl. Sed dictum quis massa in varius. Pellentesque a eleifend felis, sit amet rutrum metus. Mauris sit amet nulla lorem. In tempor quis lib

In [33]:
function AESString(str, cipherkey, encrypt=true)
    # Pad the string so that it has an integer multiple of 16 characters
    if (length(str) % 16) != 0
        N_pad = 16 - (length(str) % 16)
        str = str * (" " ^ N_pad)
    end
    
    str_vec = only.(split(str, ""))
    
    plaintext = UInt8.(reshape(str_vec, 4, :))
    n_cols = size(plaintext, 2)
    
    # Encrypt the Plaintext
    # Initialize Cipher Block
    ciphertext = zeros(UInt8, 4, n_cols)
    
    # Encrypt or Decrypt
    if encrypt
        for i = 0:((n_cols ÷ 4)-1)
            ciphertext[:,4i+1:4i+4] = AESEncrypt(plaintext[:,4i+1:4i+4], cipherkey)
        end
    else
        for i = 0:((n_cols ÷ 4)-1)
            ciphertext[:,4i+1:4i+4] = AESDecrypt(plaintext[:,4i+1:4i+4], cipherkey)
        end
    end
    
    # Return Encrypted/Decrypted String
    String(Char.(vec(ciphertext)))
end

AESString (generic function with 2 methods)

In [34]:
println(AESString(AESString(HumptyDumpty, k), k, false))

Humpty Dumpty sat on a wall.
Humpty Dumpty had a great fall.
All the king's horses and all the king's men
Couldn't put Humpty together again.   


In [35]:
strip(AESString(AESString(strip(Lorem), k),k,false)) == strip(Lorem)

true

---
---
---

## Thank you for listening!

---
---
---

### Bibliography

[1] “Advanced Encryption Standard,” Wikipedia. Mar. 05, 2023. Accessed: Mar. 06, 2023. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Advanced_Encryption_Standard&oldid=1143098771

[2] Harjoitukset, “Algebra 2: Harjoitukset 2.” Accessed: Mar. 09, 2023. [Online]. Available: https://dept.math.lsa.umich.edu/~kesmith/Alg2Demo2.pdf

[3] I. Karonen, “Answer to ‘How to do Hexadecimal multiplication in GF(2^8),’” Cryptography Stack Exchange, Oct. 16, 2018. https://crypto.stackexchange.com/a/63152 (accessed Mar. 14, 2023).

[4] “Confusion and diffusion,” Wikipedia. Jan. 09, 2023. Accessed: Mar. 14, 2023. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Confusion_and_diffusion&oldid=1132575471

[5] “Extended Euclidean algorithm,” Wikipedia. Jan. 25, 2023. Accessed: Mar. 07, 2023. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Extended_Euclidean_algorithm&oldid=1135569411

[6] “Figure 4. AES S-Box (Rijndael S-Box) 16 .,” ResearchGate. https://www.researchgate.net/figure/AES-S-Box-Rijndael-S-Box-16_fig4_318906543 (accessed Mar. 13, 2023).

[7] “Finite field arithmetic,” Wikipedia. Feb. 07, 2023. Accessed: Mar. 14, 2023. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Finite_field_arithmetic&oldid=1137942969

[8] “IBM Documentation,” Mar. 01, 2021. https://www.ibm.com/docs/en/ztpf/2020?topic=concepts-symmetric-cryptography (accessed Mar. 06, 2023).

[9] “IBM Documentation,” Mar. 01, 2021. https://www.ibm.com/docs/en/ztpf/2020?topic=concepts-symmetric-cryptography (accessed Mar. 15, 2023).

[10] “Implementing AES.” https://blog.nindalf.com/posts/implementing-aes/ (accessed Mar. 07, 2023).

[11] D. Forney, “Introduction to Finite Fields.” MIT. Accessed: Mar. 09, 2023. [Online]. Available: http://web.stanford.edu/~marykw/classes/CS250_W19/readings/Forney_Introduction_to_Finite_Fields.pdf

[12] “Lorem Ipsum - All the facts - Lipsum generator.” https://www.lipsum.com/feed/html (accessed Mar. 15, 2023).

[13] “Rijndael S-box,” Wikipedia. Sep. 14, 2022. Accessed: Mar. 07, 2023. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Rijndael_S-box&oldid=1110299033

[14] “Symmetric vs. Asymmetric Encryption - What are differences?,” SSL2BUY. https://www.ssl2buy.com/wiki/symmetric-vs-asymmetric-encryption-what-are-differences (accessed Mar. 15, 2023).

[15] J. Daemen and V. Rijmen, The design of Rijndael: AES--the Advanced Encryption Standard. Berlin ; New York: Springer, 2002.

[16] “What Is AES Encryption and How Does It Work? - Simplilearn,” Simplilearn.com, Jul. 27, 2021. https://www.simplilearn.com/tutorials/cryptography-tutorial/aes-encryption (accessed Mar. 06, 2023).

[17] “What is the Advanced Encryption Standard (AES)? Definition from SearchSecurity.” https://www.techtarget.com/searchsecurity/definition/Advanced-Encryption-Standard (accessed Mar. 15, 2023).


---
---
---
---

# Mini Homework

**Tomás Gillanders**  
**PID: A17694974**

## Problem 1: Multiplication in $GF(2^8)$

As discussed in §1.3 (Operations on Finite Fields & Polynomial Representations) of the lecture, "multiplication" in $GF(p^n)$ must be defined diferently to the familiar operation of multiplication on the real number field that we use every day, so that it satisfies the field axioms (as specified in §1.1).

In the implementation of `MixColumns` above, the function `mul` was used when multiplying elements of $GF(2^8)$.  This function was defined in the 'AESTables.jl' file but here we explore how it works.

Multiplication in $GF(2^8)$ can be carried out using a modified version of the "Peasants Algorithm" for multiplicaiton (see this [Numberphile video](https://youtu.be/HJ_PP5rqLg0) starring Johnny Ball for more information).

The multiplication algorithm is as follows (to computed $a \otimes b$ in $GF(2^8)$):
1. Initialise a variable `prod` (of type `UInt8`, i.e. stored in a byte of memory) that will updated throughout the multiplication process and result as the product of the numbers $a,b \in GF(2^8)$.
2. Writing `a` and `b` in binary representation, check if the rightmost bit ("unit" position) of `b` is a 1.  If it is add `a` to `prod`.
3. Now check if the leftmost bit of `a` is a 1 and keep track of it using the variable `overflow`.
4. Shift `a` one bit to the left and disregard the leftmost bit.  If the leftmost bit of `a` was found to be 1 in the previous step, then add $00011011_2$ to `a`.  Otherwise, do nothing.
5. Shift `b` one bit to the right, disregarding the rightmost bit and setting the leftmost bit to 0.
6. Repeat Steps 2-5 until `b = 00000000`. Return `prod`, the product of $a$ and $b$ in $GF(2^8)$.

>**Explanation**
>
>The above process works for the same reason that the "Peasant's Algorithm" for multiplication works.  Left bitshifts *in general* correspond to multiplication by 2 (with a caveat when the leftmost bit is 1) and right bitshifts correspond to integer division by 2 (or $10_2$).  Thus, by checking the rightmost bit of `b` on every iteration (while 'doubling' `a` at the same time), we are checking if the $2^n \times a$ term should be added to `prod` (where $n$ correponds to the index of the $2^n$ term in the binary representation of $b$, starting at index zero from the right).  This is exactly what we do when performing long multiplication, except performed base 10.
>
>The complication in the above algorithm arises when the leftmost bit of `a` is 1.  Since we are working over the field $GF(2^8)$, we cannot simply double a number greater than $2^7 \equiv 10000000_2$ since it will no longer be in the allowed range. 
>
>Recall that in terms of the polynomial representation of $GF(2^8)$, "doubling" corresponds to simply multiplying our polynomial by $x$.  Thus, if $deg(a(x)) = 7$ then $deg(x \bullet a(x)) = 8 \nless 8$ and thus, the resulting polynomial must be reduced by the reduction polynomial.
>
>The reduction of `a` (after the left bitshift) modulo our reduction polynomial ($m(x) = x^8 + x^4 + x^3 + x + 1$ from §1.4), corresponds to addition of `a` with $00011011_2$ (i.e. the bits correspond to the coefficients of $m(x)$ without the $x^8$ term*).  This works because if $a(x) = q(x) \bullet m(x) + r(x)$, then $a(x) \equiv r(x) \pmod(m(x))$.  However, since we know $deg(a(x)) = 8$ (after bitshift), then $a(x) = 1 \bullet m(x) + r(x) \implies r(x) = a(x) - m(x)$.  Since the bitwise XOR is its own inverse in $GF(2^8)$, we then replace `a` with `a ⊻ 00011011`, its correct value in $GF(2^8)$.

---
>\* The bit correspinding to the $x^8$ term can be omitted, since the leftmost bit of `a` is disregarded after the left bitshift.  If both of these bits were not disregarded, it would make no difference, since they would still cancel each other out: 1 ⊻ 1 = 0.

---

**Question**:

Write a function, `mul`, that takes in two elements from $GF(2^8)$, in hexadecimel format, and computes their product (in $GF(2^8)$) using the above procedure and the bitwise operators provided by Julia. The explicite binary represetations should not be used in this function.

A list of the Julia's operators can be found [here](https://docs.julialang.org/en/v1/manual/mathematical-operations/#Bitwise-Operators).

In [84]:
# ANSWER
function mul(a::UInt8, b::UInt8)
    prod::UInt8 = 0
    for _ = 1:8
        if (b & 1) != 0
            prod = prod ⊻ a
        end
        overflow = ((a & 0x80) != 0)
        a = a << 1
        if overflow
            a = a ⊻ 0x1b
        end
        b = b >> 1
    end
    prod
end

(Univariate Polynomial Ring in x over Galois field with characteristic 2, x)

x

(Univariate Polynomial Ring in x over Galois field with characteristic 2, x)

## Problem 2: Round Key Generation

As discussed in during the lecture, in order to carry out the `AddRoundKey` round transformation, a *Round Key* must be generated using the symmetric cipher key that is passed into the AES encryption function.

> *Warning*: If ever you wish to produce your own implementation of AES, it is extremely important that the `ExpandedKey` is always derived from the cipher key; and never specified directly.  Otherwise, you put your information (and possibly others') at risk from being deciphered and read by people who the message is not intended for.

The method in which the `ExpandedKey` is derived is specified in the [AES Specification](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.197.pdf).

A summary of how `KeyExpansion` works is as follows:


In [72]:
function KeyExpansionNew(K)
    N_rounds = 10
    N_key = size(K, 2)
    
    if !((N_key == 4) || (N_key == 6))
        error("Key Length Prohibited")
    end
    
    W = zeros(UInt8, 4, 4*(N_rounds+1))
    W[:,1:N_key] = K
    
    for i = (N_key+1):(4*(N_rounds+1))
        if (i % N_key) == 0
            RCon = zeros(UInt8, 4)
            RCon[1] = RC[Int(i/N_key)]
            W[:,i] = W[:,i-N_key] .⊻ SubByte.(W[:,i-1]) .⊻ RCon
        else
            W[:,i] = W[:,i-N_key] .⊻ W[:,i-1]
        end
    end
    return W
end

KeyExpansionNew (generic function with 1 method)

In [73]:
K = rand(UInt8, 4, 6)

4×6 Matrix{UInt8}:
 0x65  0xe4  0x1a  0x39  0x43  0xb9
 0xd5  0x45  0xf0  0xa5  0xca  0x68
 0xc7  0x2f  0x5d  0x09  0x95  0x54
 0x42  0x2e  0x7a  0x0f  0x21  0x16

In [74]:
KeyExpansionNew(K)

4×44 Matrix{UInt8}:
 0x65  0xe4  0x1a  0x39  0x43  0xb9  …  0x1e  0x07  0x4b  0xf8  0xe7  0x37
 0xd5  0x45  0xf0  0xa5  0xca  0x68     0x6f  0xda  0x85  0x40  0xaf  0x4f
 0xc7  0x2f  0x5d  0x09  0x95  0x54     0x9d  0x3d  0xdc  0x36  0x1a  0xe5
 0x42  0x2e  0x7a  0x0f  0x21  0x16     0xc1  0x0d  0xbc  0x6e  0xfa  0xc4

In [53]:
192/32

6.0

In [71]:
zeros(UInt8, 4)[1] = RC[Int(1)]

0x01